In [1]:
import os

In [2]:
%pwd

'e:\\Trekathon\\-Early-Fish-Disease-Detection\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Trekathon\\-Early-Fish-Disease-Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [6]:
from early_fish_disease_detection.constants import *
from early_fish_disease_detection.utils.common import read_yaml, create_dir

In [11]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dir([self.config.artifacts_root])

    
    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_dir([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [13]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [14]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

2025-01-07 12:31:19,670 - Early-Fish-Disease-Detection-logger - INFO - yaml file config\config.yaml read successfully
2025-01-07 12:31:19,677 - Early-Fish-Disease-Detection-logger - INFO - yaml file params.yaml read successfully
2025-01-07 12:31:19,677 - Early-Fish-Disease-Detection-logger - INFO - Directory artifacts created successfully
2025-01-07 12:31:20,008 - Early-Fish-Disease-Detection-logger - INFO - Directory artifacts\prepare_callbacks\checkpoint_dir created successfully
2025-01-07 12:31:20,015 - Early-Fish-Disease-Detection-logger - INFO - Directory artifacts\prepare_callbacks\tensorflow_log_dir created successfully
